# Housing Market Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

In [ ]:
# Load data into dataframe
df = pd.read_csv('NY-House-Dataset.csv')

In [ ]:
# Drop irrelevant columns
df = df.drop(columns=['LONGITUDE', 'LATITUDE','LONG_NAME','STREET_NAME','ADMINISTRATIVE_AREA_LEVEL_2','FORMATTED_ADDRESS','MAIN_ADDRESS','BROKERTITLE','LOCALITY'])
df['TYPE'] = df['TYPE'].replace('Condop for sale', 'Condo for sale')


In [ ]:
# Replace mean values with nan
df.loc[(df['BATH'] > 2) & (df['BATH'] < 3), 'BATH'] = pd.NA
df.loc[(df['PROPERTYSQFT'] > 2184) & (df['PROPERTYSQFT'] < 2185), 'PROPERTYSQFT'] = pd.NA

In [ ]:
# Prepare dataframe to predict number of bathrooms
df_bath = df.drop(columns=['PROPERTYSQFT'])
df_sqft = df.drop(columns=['BATH'])

In [ ]:
unique_values = df['TYPE'].unique()
unique_values

In [ ]:
#Encode categorical values
encoder = OrdinalEncoder(categories=[['Manhattan', 'New York County', 'Richmond County', 'Kings County',
       'New York', 'East Bronx', 'Brooklyn', 'The Bronx', 'Queens',
       'Staten Island', 'Queens County', 'Bronx County', 'Coney Island',
       'Brooklyn Heights', 'Jackson Heights', 'Riverdale', 'Rego Park',
       'Fort Hamilton', 'Flushing', 'Dumbo', 'Snyder Avenue']])
df_bath['SUBLOCALITY_ENCODED'] = encoder.fit_transform(df[['SUBLOCALITY']])

encoder = OrdinalEncoder(categories=[['Condo for sale', 'House for sale', 'Townhouse for sale',
       'Co-op for sale', 'Multi-family home for sale', 'For sale',
       'Contingent', 'Land for sale', 'Foreclosure', 'Pending',
       'Coming Soon', 'Mobile house for sale']])
df_bath['TYPE_ENCODED'] = encoder.fit_transform(df[['TYPE']])

df_bath = df_bath.drop(columns=['SUBLOCALITY','STATE','ADDRESS','TYPE'])
df_bath


# Model Training

In [ ]:
# Separate rows with NaN values
df_bath_nan = df_bath[df_bath.isna().any(axis=1)]

# Separate rows without NaN values
df_bath_without_nan = df_bath[df_bath.notna().all(axis=1)]

# KNN Model

In [ ]:
# Split into features (X) and target variable (y)
X = df_bath_without_nan.drop(columns=['BATH'])
y = df_bath_without_nan['BATH']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the KNN classifier with k=5
knn_model = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_model.predict(X_test)
y_pred

f1 = f1_score(y_test, y_pred,average='weighted', zero_division= np.nan)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred,average='weighted')
precision = precision_score(y_test, y_pred,average='weighted', zero_division= np.nan)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)

# Random Forest

In [ ]:
# Split into features (X) and target variable (y)
X = df_bath_without_nan.drop(columns=['BATH'])
y = df_bath_without_nan['BATH']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
y_pred

f1 = f1_score(y_test, y_pred,average='weighted', zero_division= np.nan)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred,average='weighted')
precision = precision_score(y_test, y_pred,average='weighted', zero_division= np.nan)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)

# Support vector machines(SVM)

In [ ]:
# Split into features (X) and target variable (y)
X = df_bath_without_nan.drop(columns=['BATH'])
y = df_bath_without_nan['BATH']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVM model with a linear kernel
svm_model = SVC(kernel='linear', random_state=42)

# Train the model
svm_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_scaled)

# Evaluate the model's accuracy
f1 = f1_score(y_test, y_pred,average='weighted', zero_division= np.nan)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred,average='weighted', zero_division= np.nan)
precision = precision_score(y_test, y_pred,average='weighted', zero_division= np.nan)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)